<a href="https://colab.research.google.com/github/Faabry/Data_Analysis/blob/main/Falecimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> <b> Web Scraping Funerário </b> </h1>
<p> Web scraping, também conhecido como raspagem de dados, é o processo de extrair informações e dados estruturados de sites da web. Isso é feito usando software automatizado, que lê o conteúdo do site e extrai as informações desejadas em um formato estruturado, como um arquivo CSV ou um banco de dados. Essas informações podem ser usadas para análise de dados, pesquisa de mercado, monitoramento de preços, entre outras finalidades. No entanto, é importante lembrar que alguns sites podem proibir ou restringir o uso de técnicas de web scraping.</p>

In [ ]:
from platform import python_version
print(f"Versão Python para esse projeto: {python_version()}")

Versão Python para esse projeto: 3.9.16


<h1> <b> Importando as bibliotecas necessárias para a execução do projeto </b> </h1>

In [ ]:
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import pandas as pd

<h1> <b> Raspagem de dados </b> </h1>

In [ ]:
# Definindo o ano através da biblioteca datetime para que o resultado seja
# 100% dinâmico em relação ao presente momento da execução do script
now = datetime.now()
year = now.year

url = "https://www.araras.sp.gov.br/falecimentos/pesquisa_falecimentos.php"

# Esse parâmetro é usado para que o usuário diga ao servidor que não é uma 
# máquina fazendo essa busca
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"}


# Criando um dicionário para armazenar as informações da página web
falecidos = {"Nome":     [],
             "Idade":    [],
             "DataFal":  [],
             "DataSep":  []}

# Criando um looping for para extrair todos os dados em um periodo de tempo
for y in range(2012, year + 1):
  for m in range(1, 13):
    data = {"mes": {m}, "ano": {y}}
    response = requests.post(url, data=data, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser", from_encoding="utf-8")

    feretros = soup.find_all("div", {"class":"col-md-9 sub-falecimento"})
    data_fal = soup.find_all("div", {"class":"col-md-7 sub-falecimento"})
    result = [i.get_text() for i in feretros]
    data_falecimento = [i.get_text() for i in data_fal]

    for k, v in enumerate(result):
        if k % 2 == 0:
            falecidos["Nome"].append(v)
        else:
            falecidos["Idade"].append(v)

    for k, v in enumerate(data_falecimento):
        if k % 2 == 0:
            falecidos["DataFal"].append(v)
        else:
            falecidos["DataSep"].append(v)

/usr/local/lib/python3.9/dist-packages/bs4/__init__.py:226: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


<h1> <b> Criando um DataFrame </b> </h1>
<p> Nosso DataFrame será criado a partir do dicionário com os dados extraídos do site </p>

In [ ]:
df = pd.DataFrame(falecidos)
df.head()

,Nome,Idade,DataFal,DataSep
0,Gilson de Jesus Tavares,"Aos 40 anos, Casado. Residia no endereço: Rua:...",31/01/2012,31/01/2012 às - Cemitério Municipal: Sepultam...
1,Luzia de Lourdes Silveira Dias,"Aos 66 anos, Viúva. Residia no endereço: Rua -...",31/01/2012,01/02/2012 às - Cemitério Municipal - Sepulta...
2,Luci Terezinha Martim De Paula,"Aos 53 anos, Casada. Residia no endereço: Rua ...",30/01/2012,30/01/2012 às - Cemitério Municipal - Sepulta...
3,Valter Sorrantino,"Aos 47 anos, Casado. Residia no endereço: Rua ...",30/01/2012,31/01/2012 às - Cemitério Municipal - Sepulta...
4,Antonio Vieira dos Santos,"Aos 55 anos, Solteiro. Residia no endereço: Ru...",30/01/2012,31/01/2012 às - Cemitério Municipal: Sepultam...


# **Tratando os dados**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10056 entries, 0 to 10055
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Nome     10056 non-null  object
 1   Idade    10056 non-null  object
 2   DataFal  10056 non-null  object
 3   DataSep  10056 non-null  object
dtypes: object(4)
memory usage: 314.4+ KB


<p> Não temos nenhum valor ausente, mas os tipos primitivos de algumas colunas como <b> DataFal e DataSep </b> estão incorretos.

<h1> <b> Task 1:</b> Verificar se existem valores duplicados</h1>

In [ ]:
# O primerio passo é remover possíveis espaços em branco que podem indicar valores
# diferentes ao pandas
df = df.apply(lambda x: x.str.strip())

In [ ]:
df[["Nome", "Idade"]].duplicated().sum()

17

In [ ]:
df.loc[df["Nome"] == "Jose Abilio Baggio"]

,Nome,Idade,DataFal,DataSep
8634,Jose Abilio Baggio,"Aos 95 anos, Casado. Residia no endereço: Rua ...",28/11/2021,28/11/2021 às 16:00 - CEMITÉRIO MUNICIPAl -SP.
8635,Jose Abilio Baggio,"Aos 95 anos, Casado. Residia no endereço: Rua ...",28/11/2021,28/11/2021 às 16:00 - CEMITÉRIO MUNICIPAl -SP.
8636,Jose Abilio Baggio,"Aos 95 anos, Casado. Residia no endereço: Rua ...",28/11/2021,28/11/2021 às 16:00 - CEMITÉRIO MUNICIPAl -SP.
8637,Jose Abilio Baggio,"Aos 95 anos, Casado. Residia no endereço: Rua ...",28/11/2021,28/11/2021 às 16:00 - CEMITÉRIO MUNICIPAl - ...


<p> Como podemos ver temos alguns valores duplicados, o próximo passo é remove-los </p>

In [ ]:
df.drop_duplicates(["Nome", "Idade"], inplace=True)

In [ ]:
df.count()

Nome       10039
Idade      10039
DataFal    10039
DataSep    10039
dtype: int64

In [ ]:
df.head()

,Nome,Idade,DataFal,DataSep
0,Gilson de Jesus Tavares,"Aos 40 anos, Casado. Residia no endereço: Rua:...",31/01/2012,31/01/2012 às - Cemitério Municipal: Sepultam...
1,Luzia de Lourdes Silveira Dias,"Aos 66 anos, Viúva. Residia no endereço: Rua -...",31/01/2012,01/02/2012 às - Cemitério Municipal - Sepulta...
2,Luci Terezinha Martim De Paula,"Aos 53 anos, Casada. Residia no endereço: Rua ...",30/01/2012,30/01/2012 às - Cemitério Municipal - Sepulta...
3,Valter Sorrantino,"Aos 47 anos, Casado. Residia no endereço: Rua ...",30/01/2012,31/01/2012 às - Cemitério Municipal - Sepulta...
4,Antonio Vieira dos Santos,"Aos 55 anos, Solteiro. Residia no endereço: Ru...",30/01/2012,31/01/2012 às - Cemitério Municipal: Sepultam...


<h1> <b>Task 2:</b> Tratar os dados para que se tornem informações</h1>

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
df["Local"] = df["DataSep"].apply(lambda x: x[10::])
df["DataSep"] = df["DataSep"].apply(lambda x: x[0: 11])
df["Idade"] = df["Idade"].apply(lambda x: x[3:7])
df.head()

,Nome,Idade,DataFal,DataSep,Local
0,Gilson de Jesus Tavares,40,31/01/2012,31/01/2012,às - Cemitério Municipal: Sepultamento as 10...
1,Luzia de Lourdes Silveira Dias,66,31/01/2012,01/02/2012,às - Cemitério Municipal - Sepultamento ás 0...
2,Luci Terezinha Martim De Paula,53,30/01/2012,30/01/2012,às - Cemitério Municipal - Sepultamento ás 1...
3,Valter Sorrantino,47,30/01/2012,31/01/2012,às - Cemitério Municipal - Sepultamento ás 0...
4,Antonio Vieira dos Santos,55,30/01/2012,31/01/2012,às - Cemitério Municipal: Sepultamento as 10...


In [ ]:
df["Idade"] = df["Idade"].str.strip()

In [ ]:
df["Idade"].unique()

array(['40', '66', '53', '47', '55', '62', '90', '88', '32', '81', '79',
       '24', '34', '76', '82', '80', '89', '49', '31', '73', '78', '65',
       '96', '74', '42', '7 a', '94', '57', '60', '68', '41', '50', '48',
       '51', '77', '84', '75', '69', '39', '44', '56', '46', '64', '92',
       '21', '52', '43', '72', '59', '58', '67', '54', '61', '87', '63',
       '85', '93', '95', '45', '91', '83', '86', '97', '70', '29', '71',
       '26', '3 a', '33', '35', '36', '27', '22', '19', '38', '28', '37',
       '101', '23', '18', '103', '98', '102', '16', '100', '20', '25',
       '99', '0 a', '14', '17', '12', '105', '30', '104', '4 a', '13',
       '2 a', '15', '8 a', '6 a', '9 a', '5 a', '107', '11', '106'],
      dtype=object)

<p> Existem alguns valores com strings como <b>5 a</b> isso será um problema na análise final e precisa ser resolvido. </p>

In [ ]:
# Substituindo valores "_ a" por números
for c in range(0, 10):
  df["Idade"] = df["Idade"].str.replace(f"{c} a", f"{c}")

In [ ]:
df["Idade"].unique()

array(['40', '66', '53', '47', '55', '62', '90', '88', '32', '81', '79',
       '24', '34', '76', '82', '80', '89', '49', '31', '73', '78', '65',
       '96', '74', '42', '7', '94', '57', '60', '68', '41', '50', '48',
       '51', '77', '84', '75', '69', '39', '44', '56', '46', '64', '92',
       '21', '52', '43', '72', '59', '58', '67', '54', '61', '87', '63',
       '85', '93', '95', '45', '91', '83', '86', '97', '70', '29', '71',
       '26', '3', '33', '35', '36', '27', '22', '19', '38', '28', '37',
       '101', '23', '18', '103', '98', '102', '16', '100', '20', '25',
       '99', '0', '14', '17', '12', '105', '30', '104', '4', '13', '2',
       '15', '8', '6', '9', '5', '107', '11', '106'], dtype=object)

<h1> <b>Task 3:</b> Tratar as colunas de datas e converte-las para o tipo primitivo correto.</b></h1>

In [ ]:
df["DataFal"] = df["DataFal"].str.replace("/", "-")
df["DataSep"] = df["DataSep"].str.replace("/", "-")

<p> Verificando se existem valores estranhos em relação ao ano.</p>

In [ ]:
# Verificando valores ausentes
df["DataSep"].apply(lambda x: x[6::]).unique()

array(['2012 ', '2011 ', '2023 ', '2013 ', '2014 ', '0123 ', '2015 ',
       '2016 ', '2000 ', '2017 ', '2018 ', '2019 ', '2020 ', '2021 ',
       '2022 ', '2200 '], dtype=object)

<p> Foi observado que existem valores numéricos porém que não correspondem à uma data como por exemplo <b>"0123, 2000, 2200"</b>, será necessário verificar à que ano esses valores estão relacionados. </p>

In [ ]:
df.loc[df["DataSep"].str.contains("2000 ")] #2000 = 2016
df.loc[df["DataSep"].str.contains("2200 ")] #2200 = 2022
df.loc[df["DataSep"].str.contains("0123 ")] #0123 = 2014

,Nome,Idade,DataFal,DataSep,Local
1831,Anna Gonçalves félix,79,12-03-2014,13-03-0123,às 10:00 - Cemitério Municipal.


In [ ]:
df["DataSep"] = df["DataSep"].str.strip()

<h3> <b>Substituindo os valores incorretos para o seu devido ano.</b> </h3>

In [ ]:
df["DataSep"] = df["DataSep"].str.replace("2000", "2016")
df["DataSep"] = df["DataSep"].str.replace("2200", "2022")
df["DataSep"] = df["DataSep"].str.replace("0123", "2014")

In [ ]:
pd.to_datetime(df["DataFal"], format='%d-%m-%Y')
pd.to_datetime(df["DataSep"], format='%d-%m-%Y')

0       2012-01-31
1       2012-02-01
2       2012-01-30
3       2012-01-31
4       2012-01-31
5       2012-01-29
6       2012-01-27
7       2012-01-26
8       2012-01-26
9       2012-01-27
10      2012-01-25
11      2012-01-24
12      2012-01-25
13      2012-01-25
14      2012-01-23
15      2012-01-23
16      2012-01-23
17      2012-01-24
18      2012-01-24
19      2012-01-24
20      2012-01-24
21      2012-01-23
22      2012-01-23
23      2012-01-22
24      2012-01-22
25      2012-01-19
26      2012-01-19
27      2012-01-19
28      2012-01-20
29      2012-01-18
30      2012-01-18
31      2012-01-18
32      2012-01-19
33      2012-01-19
34      2012-01-19
35      2012-01-17
36      2012-01-16
37      2012-01-16
38      2012-01-14
39      2012-01-14
40      2012-01-14
41      2012-01-14
42      2012-01-14
43      2012-01-11
44      2012-01-11
45      2012-01-12
46      2012-01-11
47      2012-01-11
48      2012-01-11
49      2012-01-11
50      2012-01-11
51      2012-01-10
52      2012

In [ ]:
df["DataSep"].apply(lambda x: x[6::]).unique()

array(['2012', '2011', '2023', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019', '2020', '2021', '2022'], dtype=object)

In [ ]:
df.describe()

,Nome,Idade,DataFal,DataSep,Local
count,10039,10039,10039,10039,10039
unique,9995,106,3672,3711,938
top,Antonio Ferreira,83,26-06-2013,13-02-2014,às 16:00 - Cemitério Municipal.
freq,3,266,10,10,937


<h1> <b>Task 4:</b> Verificar se há muita diferença entre a contagem de valores para <b>["DataFal"]</b> em relação à <b>["DataSep"].</b> </h>

In [ ]:
# Contagem de valores para "ano" da coluna "DataSep"
sep = df["DataSep"].apply(lambda x: x[6::]).value_counts()

In [ ]:
# Contagem de valores para "ano" da coluna "DataFal"
fal = df["DataFal"].apply(lambda x: x[6::]).value_counts()

<p> Na coluna <b>["DataFal"]</b> existe um valor de <i>2011</i>, pois a pessoa faleceu no ultimo dia do ano e foi sepultada no primeiro dia do ano seguinte, porém se mantermos esse valor vamos ter problemas, pois para criar um DataFrame é necessário que todas as colunas tenham o mesmo tamanho, então nesse caso como é apenas um valor vamos simplesmete remove-lo.</p>

In [ ]:
a = pd.Series(sep)
b = pd.Series(fal)

# Removendo o valor de 2011
a = a.drop(index="2011")

In [ ]:
# Criando um dicionário para armazenar as informações de cada ano em relação à
# data de falecimento/sepultamento
data_dict = {"sep": a,
             "fal": b}
result = pd.DataFrame(data_dict)
result["Diferença"] = result["fal"] - result["sep"]
result

,sep,fal,Diferença
2021,1137,1132,-5
2022,1062,1066,4
2020,909,910,1
2017,882,884,2
2019,869,871,2
2015,852,849,-3
2018,850,849,-1
2012,836,836,0
2014,817,820,3
2013,803,804,1


<p> Aparentemente os valores estão normais, uma vez que é completamente normal uma pessoa falecer no ultimo dia do ano e ser sepultada no primeiro dia do ano seguinte. </p>

<h1> <b>Task 5:</b> Ajustar as informações da coluna <b>["Local"]</b>. </h1>

In [ ]:
# Removendo todo e qualquer espaço em branco no inicio ou fim da linha
# em seguida estou colocando todos os caracteres em letra minúscula para que não
# tenhamos problemas com pAlaVRas eScritAS assim 
df["Local"] = df["Local"].str.strip().str.lower()

In [ ]:
df.head()

,Nome,Idade,DataFal,DataSep,Local
0,Gilson de Jesus Tavares,40,31-01-2012,31-01-2012,às - cemitério municipal: sepultamento as 10:...
1,Luzia de Lourdes Silveira Dias,66,31-01-2012,01-02-2012,às - cemitério municipal - sepultamento ás 09...
2,Luci Terezinha Martim De Paula,53,30-01-2012,30-01-2012,às - cemitério municipal - sepultamento ás 15...
3,Valter Sorrantino,47,30-01-2012,31-01-2012,às - cemitério municipal - sepultamento ás 08...
4,Antonio Vieira dos Santos,55,30-01-2012,31-01-2012,às - cemitério municipal: sepultamento as 10:...


<p> Após uma análise foi verificado que existem muitas variações da mesma palavra então será necessário fazer o tratamento de cada uma das variações para que exista apenas um padrão. </p>

In [ ]:
for c in range(7, 10):
  df["Local"] = df["Local"].str.replace(f"às 0{c}:00 - cemitério municipal.", f"0{c}:00 Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às 0{c}:30 - cemitério municipal.", f"0{c}:30Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às  - cemitério municipal: sepultamento as 0{c}:00 hrs.", "Cemitério Municipal de Araras - SP")
  
for i in range(10, 18):
  df["Local"] = df["Local"].str.replace(f"às  - cemitério municipal: sepultamento as {i}:00 hrs.", "Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às  - cemitério Municipal: sepultamento as {i}:30 hrs.", "Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às  - cemitério Municipal: sepultamento as {i}:00 h.", "Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às  - cemitério Municipal: sepultamento as {i}:30 h.", "Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às {i}:00 - cemitério municipal.", f"{i}:00 - Cemitério Municipal de Araras - SP")
  df["Local"] = df["Local"].str.replace(f"às {i}:30 - cemitério municipal.", f"{i}:30 - Cemitério Municipal de Araras - SP")

<ipython-input-32-12cfe0309bb9>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(f"às 0{c}:00 - cemitério municipal.", f"0{c}:00 Cemitério Municipal de Araras - SP")
<ipython-input-32-12cfe0309bb9>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(f"às 0{c}:30 - cemitério municipal.", f"0{c}:30Cemitério Municipal de Araras - SP")
<ipython-input-32-12cfe0309bb9>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(f"às  - cemitério municipal: sepultamento as 0{c}:00 hrs.", "Cemitério Municipal de Araras - SP")
<ipython-input-32-12cfe0309bb9>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(f"às  - cemitério municipal: sepultamento as {i}:00 hrs."

In [ ]:
df["Local"] = df["Local"].str.replace("cemitério municipal de araras sp.",'Cemitério Municipal de Araras - SP')\
                         .str.replace("às cemitério municipal de araras - sp", "Cemitério Municipal de Araras - SP")\
                         .str.replace("cemitério municipal de araras - spde araras - sp", "Cemitério Municipal de Araras - SP")\
                         .str.replace("às-cemitério municipal.", "Cemitério Municipal de Araras - SP")\
                         .str.replace("às-cemitériomunicipaldeararahoarioadefinirs-sp.", "Cemitério Municipal de Araras - SP")\
                         .str.replace("às-cemitériopq.doseucaliptos-araras-sp.", "Cemitério Pq dos Eucaliptos")\
                         .str.replace("às-crematóriomonumentalemlimeira-sp.", "Crematório Monumental")\
                         .str.replace("às-crematóriopq.daspalmeiras-rioclaro-sp.", "Crematório Pq Das Palmeiras - Rio Claro - SP")\
                      

<ipython-input-33-18b0d580ede4>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace("cemitério municipal de araras sp.",'Cemitério Municipal de Araras - SP')\


# Horarios


In [ ]:
# Criando uma nova coluna para receber a informação do horário do sepultamento
df["Horário"] = df["Local"].apply(lambda x: x[0:5]).str.lower()
df.head()

,Nome,Idade,DataFal,DataSep,Local,Horário
0,Gilson de Jesus Tavares,40,31-01-2012,31-01-2012,às - cemitério municipal: sepultamento as 10:...,às -
1,Luzia de Lourdes Silveira Dias,66,31-01-2012,01-02-2012,às - cemitério municipal - sepultamento ás 09...,às -
2,Luci Terezinha Martim De Paula,53,30-01-2012,30-01-2012,às - cemitério municipal - sepultamento ás 15...,às -
3,Valter Sorrantino,47,30-01-2012,31-01-2012,às - cemitério municipal - sepultamento ás 08...,às -
4,Antonio Vieira dos Santos,55,30-01-2012,31-01-2012,às - cemitério municipal: sepultamento as 10:...,às -


In [ ]:
df["Horário"].unique()

array(['às  -', '14:30', 'cemit', '16:00', '16:30', '13:00', '13:30',
       '14:00', 'às 15', '15:30', '10:30', '15:00', '09:30', '10:00',
       'às 00', '08:00', 'às 90', 'às 83', '08:30', 'às 16', '09:00',
       'às 10', 'às 14', 'às 13', '11:00', '17:00', '12:00', 'às 12',
       'às 17', 'às 08', 'às 09', '11:30', 'às 18', '17:30', 'às 8:',
       '12:30', 'às 11', 'às 07', '07:30', 'às 19', 'às 06', 'às 80',
       '07:00', 'às 05'], dtype=object)

<p> Existem alguns valores não correspondentes à um valor de horas, portanto será necessário tratá-los. </p>

In [ ]:
mask = df['Horário'].str.startswith('às  -')

# extract last 5 elements of local column for filtered rows
filtered_df = df.loc[mask, 'Local'].str[-10:]

In [ ]:
df.loc[mask, "Horário"] = filtered_df.values

In [ ]:
df["Horário"] = df["Horário"].str.replace("s", "")
df["Horário"] = df["Horário"].str.replace("h.", "")
df["Horário"] = df["Horário"].str.replace("hrs.", "")
df["Horário"] = df["Horário"].str.replace(".", "")
df["Horário"] = df["Horário"].str.replace("a", "")
df["Horário"] = df["Horário"].str.replace("á", "")
df["Horário"] = df["Horário"].str.replace("à", "")


<ipython-input-38-c4aca1d63508>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Horário"] = df["Horário"].str.replace("h.", "")
<ipython-input-38-c4aca1d63508>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Horário"] = df["Horário"].str.replace("hrs.", "")
<ipython-input-38-c4aca1d63508>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["Horário"] = df["Horário"].str.replace(".", "")


In [ ]:
df["Horário"] = df["Horário"].str.strip()

In [ ]:

df["Horário"].unique()

array(['10:30', '09:30', ':00', '08:00', '10:00', '16:30', '16:00',
       '15:30', '14:00', '9:00', '08:30', '13:30', '14:30', '13:00',
       '9:30', '15:00', '16:45', 'te di', '11:30', '160', '8:30', '3:00',
       '8:00', 'municipl', 'o  80', 'to  10', '100', '09:00', '17 r',
       'o  140', '6:30', 'uclipto', 'o 10:00', ':30', ':30 r', 'cemit',
       '4:30', '0:00', 'n b', '8/04/2012', '0:30', 'm rr', '5:30',
       '16:20', 'e de leme', '15', '00', 'formo', 'lvdo-p', '90', '83',
       '16', '10', '14', '13', '11:00', '17:00', '12:00', '12', '17',
       '08', '09', '18', 'ão  preto', '17:30', '8:', 'o rel/b', '12:30',
       '11', 'e concl', 'cmpin', '07', 'mn )', '07:30', 'remtorio', '19',
       '06', 'm limeir', 'emtorio', 'o pulo', '80', '07:00', 'crlo )',
       'de buru', 'de rr', 'plmeir', '05', 'nir - p', 'rr -p', 'meir -p',
       'clro-p', 'ão pulo'], dtype=object)

In [ ]:
import numpy as np
np.where(pd.to_datetime(df["Horário"], format='%H:%M', errors='coerce').isna(), '00:00:00', df["Horário"])

array(['10:30', '09:30', '00:00:00', ..., '13:00', '14:00', '09:00'],
      dtype=object)

In [ ]:
df["Horário"] = df["Horário"].str.replace("116:30", "16:30")
df["Horário"] = df["Horário"].str.replace("115:30", "15:30")
df["Horário"] = df["Horário"].str.replace("113:00", "13:00")
df["Horário"] = df["Horário"].str.replace("te di", "13:00")
df["Horário"] = df["Horário"].str.replace("uclipto", "15:00")
df["Horário"] = df["Horário"].str.replace("8/04/2012", "16:00")
df["Horário"] = df["Horário"].str.replace("e de leme", "10:00")

In [ ]:
df["Horário"].unique()

array(['10:30', '09:30', ':00', '08:00', '10:00', '16:30', '16:00',
       '15:30', '14:00', '9:00', '08:30', '13:30', '14:30', '13:00',
       '9:30', '15:00', '16:45', '11:30', '160', '8:30', '3:00', '8:00',
       'municipl', 'o  80', 'to  10', '100', '09:00', '17 r', 'o  140',
       '6:30', 'o 10:00', ':30', ':30 r', 'cemit', '4:30', '0:00', 'n b',
       '0:30', 'm rr', '5:30', '16:20', '15', '00', 'formo', 'lvdo-p',
       '90', '83', '16', '10', '14', '13', '11:00', '17:00', '12:00',
       '12', '17', '08', '09', '18', 'ão  preto', '17:30', '8:',
       'o rel/b', '12:30', '11', 'e concl', 'cmpin', '07', 'mn )',
       '07:30', 'remtorio', '19', '06', 'm limeir', 'emtorio', 'o pulo',
       '80', '07:00', 'crlo )', 'de buru', 'de rr', 'plmeir', '05',
       'nir - p', 'rr -p', 'meir -p', 'clro-p', 'ão pulo'], dtype=object)

# Local

In [ ]:
df["Local"] = df["Local"].apply(lambda x: x[11::])

In [ ]:
df["Local"] = df["Local"].str.lower()

In [ ]:
araras = df["Local"].loc[df["Local"].str.contains("ério municipal")]
araras.count()

8622

In [ ]:
pq = df["Local"].loc[df["Local"].str.contains("eucalipt")]
pq2 = df["Local"].loc[df["Local"].str.contains("eucalíp")]
pq.count()

434

In [ ]:
leme = df["Local"].loc[df["Local"].str.contains("leme")]
leme.count()

23

In [ ]:
monumental = df["Local"].loc[df["Local"].str.contains("monumental")]
monumental.count()

26

In [ ]:
limeira = df["Local"].loc[df["Local"].str.contains("limeira")]
limeira.count()

44

In [ ]:
conchal = df["Local"].loc[df["Local"].str.contains("conchal")]
conchal.count()

36

In [ ]:
rio_claro = df["Local"].loc[df["Local"].str.contains("rio claro")]
rio_claro.count()

26

In [ ]:
import re
for row in pq.values:
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério pq. dos eucalíptos araras sp")

for row in pq2.values:
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério pq. dos eucalíptos araras sp")

<ipython-input-53-6d720c1e0d68>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério pq. dos eucalíptos araras sp")
<ipython-input-53-6d720c1e0d68>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério pq. dos eucalíptos araras sp")


In [ ]:
for row in araras.values:
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério municipal de araras sp")

<ipython-input-54-9309985adbfe>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério municipal de araras sp")


In [ ]:
for row in leme.values:
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério municipal de leme sp")

<ipython-input-55-efc9bf87020a>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério municipal de leme sp")


In [ ]:
for row in monumental.values:
  df["Local"] = df["Local"].str.replace(re.escape(row), "crematório monumental bom pastor")

for row in limeira.values:
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério municipal de limeira sp")

for row in conchal.values:
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério municipal de conchal sp")

for row in rio_claro.values:
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério municipal de rio claro sp")

<ipython-input-56-622b2087c9ac>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(re.escape(row), "crematório monumental bom pastor")
<ipython-input-56-622b2087c9ac>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério municipal de limeira sp")
<ipython-input-56-622b2087c9ac>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério municipal de conchal sp")
<ipython-input-56-622b2087c9ac>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério municipal de rio claro sp")


In [ ]:
# df["Local"] = df["Local"].str.replace("cemitério municipal de araras spde araras - sp.", "cemitério municipal de araras sp")
# df["Local"] = df["Local"].str.replace("cemitério municipal de araras spde araras sp.", "cemitério municipal de araras sp")
# df["Local"] = df["Local"].str.replace("cemitério municipal de araras spde araras.", "cemitério municipal de araras sp")
# df["Local"] = df["Local"].str.replace("cemitério municipal de araras spde araras / sp", "cemitério municipal de araras sp")
# df["Local"] = df["Local"].str.replace("cemitério municipal de araras spararas.", "cemitério municipal de araras sp")
# df["Local"] = df["Local"].str.replace("cemitério municipal de araras sp/ sp.", "cemitério municipal de araras sp")
# df["Local"] = df["Local"].str.replace("cemitério municipal.", "cemitério municipal de araras sp")
# df["Local"] = df["Local"].str.replace("cemitério municipal de araras spde araras sp", "cemitério municipal de araras sp")
# df["Local"] = df["Local"].str.replace("c.", "cemitério municipal de araras sp")
# df["Local"] = df["Local"].str.replace("ce.", "cemitério municipal de araras sp")

In [ ]:
araras2 = df["Local"].loc[df["Local"].str.contains("rio municipal de araras")]
araras2.count()

9347

In [ ]:
for row in araras2.values:
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério municipal de araras sp")

<ipython-input-59-138c1f9da44d>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Local"] = df["Local"].str.replace(re.escape(row), "cemitério municipal de araras sp")


In [ ]:
df["Local"].describe()

count                                10039
unique                                 131
top       cemitério municipal de araras sp
freq                                  9342
Name: Local, dtype: object

In [ ]:
df.head()

,Nome,Idade,DataFal,DataSep,Local,Horário
0,Gilson de Jesus Tavares,40,31-01-2012,31-01-2012,cemitério municipal de araras sp,10:30
1,Luzia de Lourdes Silveira Dias,66,31-01-2012,01-02-2012,cemitério municipal de araras sp,09:30
2,Luci Terezinha Martim De Paula,53,30-01-2012,30-01-2012,cemitério municipal de araras sp,:00
3,Valter Sorrantino,47,30-01-2012,31-01-2012,cemitério municipal de araras sp,08:00
4,Antonio Vieira dos Santos,55,30-01-2012,31-01-2012,cemitério municipal de araras sp,10:00


In [ ]:
df.to_excel("falecimentos.xlsx", index=False)

In [ ]:
upload_url = 'https://onedrive.live.com/download?cid=D374F7659AEBF036&resid=D374F7659AEBF036%217030&authkey=ALTeqmYcb4ToSjk&em=excel'

# Set the request headers with your access token
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36\
(KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"}

# Convert the DataFrame to CSV and upload to OneDrive
new_file = df.to_excel("óbitos.xlsx", index=False)
response = requests.put(upload_url, headers=headers, data=new_file)

type(response)

requests.models.Response

In [ ]:
import sqlite3

In [ ]:
cnn = sqlite3.connect("óbitos.db")

In [ ]:
df.to_sql("pacientes1", cnn, index=False)

10039

# End


In [ ]:
%reload_ext sql

In [ ]:
%sql sqlite:///óbitos.db

In [ ]:
%%sql

SELECT * FROM pacientes1
LIMIT 10;

 * sqlite:///óbitos.db
Done.


Nome,Idade,DataFal,DataSep,Local,Horário
Gilson de Jesus Tavares,40,31-01-2012,31-01-2012,cemitério municipal de araras sp,10:30
Luzia de Lourdes Silveira Dias,66,31-01-2012,01-02-2012,cemitério municipal de araras sp,09:30
Luci Terezinha Martim De Paula,53,30-01-2012,30-01-2012,cemitério municipal de araras sp,:00
Valter Sorrantino,47,30-01-2012,31-01-2012,cemitério municipal de araras sp,08:00
Antonio Vieira dos Santos,55,30-01-2012,31-01-2012,cemitério municipal de araras sp,10:00
Sergio Zago,62,29-01-2012,29-01-2012,cemitério municipal de araras sp,16:30
Antonio Batistela,90,27-01-2012,27-01-2012,cemitério municipal de araras sp,16:00
Josephina Zorzenon Bianchini,88,26-01-2012,26-01-2012,cemitério municipal de araras sp,15:30
Marcia Maria Moura Lourenço,32,26-01-2012,26-01-2012,cemitério municipal de araras sp,14:00
Amelia Franzini de Lima,81,26-01-2012,27-01-2012,cemitério municipal de araras sp,9:00


In [ ]:
%%sql

SELECT * FROM pacientes1
WHERE Local LIKE "%Cemitério Municipal%"
LIMIT 10;

 * sqlite:///óbitos.db
Done.


Nome,Idade,DataFal,DataSep,Local,Horário
Gilson de Jesus Tavares,40,31-01-2012,31-01-2012,cemitério municipal de araras sp,10:30
Luzia de Lourdes Silveira Dias,66,31-01-2012,01-02-2012,cemitério municipal de araras sp,09:30
Luci Terezinha Martim De Paula,53,30-01-2012,30-01-2012,cemitério municipal de araras sp,:00
Valter Sorrantino,47,30-01-2012,31-01-2012,cemitério municipal de araras sp,08:00
Antonio Vieira dos Santos,55,30-01-2012,31-01-2012,cemitério municipal de araras sp,10:00
Sergio Zago,62,29-01-2012,29-01-2012,cemitério municipal de araras sp,16:30
Antonio Batistela,90,27-01-2012,27-01-2012,cemitério municipal de araras sp,16:00
Josephina Zorzenon Bianchini,88,26-01-2012,26-01-2012,cemitério municipal de araras sp,15:30
Marcia Maria Moura Lourenço,32,26-01-2012,26-01-2012,cemitério municipal de araras sp,14:00
Amelia Franzini de Lima,81,26-01-2012,27-01-2012,cemitério municipal de araras sp,9:00


In [ ]:
query = cnn.execute("""SELECT * FROM pacientes1
                       WHERE Local LIKE "%Cemitério Municipal%"
                       LIMIT 10;""")

In [ ]:
data = pd.DataFrame(query.fetchall())